In [ ]:
# 비디오 위험물 탐지 및 흐림 처리 시스템 (최적화 버전)
# 视频危险物体检测与模糊处理系统（优化版）
# Google Colab에서 실행 가능하며，위험물 탐지、흐림 처리 및 설명 생성 기능을 구현합니다.
# 可在Google Colab中运行，实现危险物体检测、模糊处理与解释生成功能。

# 필요한 라이브러리 설치 (학습 및 추론에 필요한 패키지 설치)
# 安装必要库（安装训练与推理所需依赖包）
!pip install torch torchvision opencv-python matplotlib moviepy transformers
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pycocotools  # COCO 데이터셋 평가 지표 추가 (COCO数据集评估指标补充)
!apt-get install -y ffmpeg  # 비디오 코덱 지원 (视频编解码器支持)

# 필요한 라이브러리 임포트 (핵심 기능 모듈 불러오기)
# 导入必要库（加载核心功能模块）
import torch  # 파이토치 프레임워크 (PyTorch框架)
import torchvision  # 컴퓨터 비전 전용 라이브러리 (计算机视觉专用库)
from torchvision import transforms  # 이미지 전처리 도구 (图像预处理工具)
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights  # 객체 탐지 모델 (目标检测模型)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor  # 모델 분류기 교체용 (模型分类器替换用)
import cv2  # OpenCV 라이브러리 (이미지/비디오 처리) (OpenCV库，用于图像/视频处理)
import numpy as np  # 수치 연산 라이브러리 (数值计算库)
import matplotlib.pyplot as plt  # 시각화 도구 (可视化工具)
from moviepy.editor import VideoFileClip  # 비디오 편집 라이브러리 (视频编辑库)
import os  # 파일 시스템 관리 (文件系统管理)
from tqdm import tqdm  # 진행 상황 바 표시 (显示进度条)
from google.colab import files  # Colab 파일 업로드/다운로드 (Colab文件上传/下载)
from transformers import pipeline  # 텍스트 생성 모델 파이프라인 (文本生成模型流水线)
from torch.utils.data import DataLoader  # 데이터 배치 처리 (数据批量处理)
from torch.utils.tensorboard import SummaryWriter  # 학습 과정 시각화 (训练过程可视化)
import json  # JSON 파일 처리 (JSON文件处理)
from datetime import datetime  # 시간 기반 로그 관리 (基于时间的日志管理)

# 디바이스 설정 (GPU 사용 가능 시 GPU 우선 적용)
# 设备设置（优先使用GPU，无GPU则使用CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")  # 사용 중인 디바이스 출력 (打印当前使用的设备)
print(f"使用设备: {device}")

# 위험물 카테고리 정의 (COCO 데이터셋 라벨 ID 기준)
# 危险物体类别定义（基于COCO数据集标签ID）
DANGEROUS_CLASSES = {
    41: "칼",  # COCO 데이터셋의 "knife"에 해당 (对应COCO数据集的"knife")
    42: "포크",  # 잠재적 위험물로 분류 (归为潜在危险物体)
    62: "총",   # COCO 데이터셋의 "gun"에 해당 (对应COCO数据集的"gun")
    # 필요에 따라 추가 위험물 카테고리 확장 가능 (可根据需求扩展更多危险物体类别)
}

# 학습 파라미터 설정 (모델 학습에 필요한 하이퍼파라미터)
# 训练参数设置（模型训练所需超参数）
TRAIN_PARAMS = {
    'lr': 0.005,          # 학습률 (学习率)
    'momentum': 0.9,      # 모멘텀 (动量，加速梯度下降)
    'weight_decay': 0.0005,# 가중치 감쇠 (防止过拟合)
    'lr_step_size': 3,    # 학습률 감소 주기 (学习率衰减周期)
    'lr_gamma': 0.1,      # 학습률 감소 비율 (学习率衰减比例)
    'num_epochs': 10,     # 총 학습 에폭 수 (总训练轮次)
    'batch_size': 2,      # 배치 크기 (批量大小)
    'workers': 2          # 데이터 로딩 워커 수 (数据加载线程数)
}

# 전처리 함수 정의 (이미지 변환 로직)
# 预处理函数定义（图像转换逻辑）
def get_transform(train):
    """이미지 전처리 변환을 가져오는 함수
    이미지 전처리 변환을 생성하는 함수로，학습 시 데이터 확장을 추가합니다.

    Args:
        train: 학습 모드 여부 플래그 (True=학습, False=추론)
        train: 训练模式标识（True=训练，False=推理）

    Returns:
        transforms.Compose: 이미지에 적용할 변환 파이프라인
        transforms.Compose: 应用于图像的转换流水线
    """
    transforms_list = []
    # 이미지를 텐서로 변환 (필수 전처리)
    # 将图像转换为张量（必需预处理步骤）
    transforms_list.append(transforms.ToTensor())

    if train:
        # 학습 시에만 데이터 확장 적용 (랜덤 좌우 반전)
        # 仅在训练时应用数据增强（随机左右翻转）
        transforms_list.append(transforms.RandomHorizontalFlip(0.5))

    return transforms.Compose(transforms_list)

# 학습용 데이터셋 클래스 (COCO 형식 데이터셋을 위한 클래스)
# 训练用数据集类（适用于COCO格式数据集的类）
class DangerousObjectsDataset(torch.utils.data.Dataset):
    """위험물 탐지를 위한 데이터셋 클래스
    COCO 형식의 주석(annotation)을 사용하는 위험물 이미지 데이터셋입니다.

    COCO格式标注的危险物体图像数据集类。
    """
    def __init__(self, root, annotations_file, transforms=None):
        """데이터셋 초기화 메서드
        데이터셋의 이미지 경로、주석 파일 및 전처리 변환을 설정합니다.

        数据集初始化方法
        设置数据集的图像路径、标注文件及预处理转换。

        Args:
            root: 이미지 파일이 저장된 루트 디렉토리 경로
            root: 图像文件存储的根目录路径
            annotations_file: COCO 형식 주석 파일 경로
            annotations_file: COCO格式标注文件路径
            transforms: 이미지에 적용할 전처리 변환 (None=변환 없음)
            transforms: 应用于图像的预处理转换（None=无转换）
        """
        self.root = root
        self.transforms = transforms

        # 주석 파일 로드 (JSON 형식)
        # 加载标注文件（JSON格式）
        with open(annotations_file, 'r') as f:
            self.annotations = json.load(f)

        self.images = self.annotations['images']  # 이미지 정보 리스트 (图像信息列表)
        self.annotations_dict = self._create_annotations_dict()  # 이미지 ID별 주석 매핑 (按图像ID映射标注)

    def _create_annotations_dict(self):
        """이미지 ID별로 주석을 정리하는 보조 함수
        이미지 ID를 키로，해당 이미지의 주석 리스트를 값으로 하는 딕셔너리를 생성합니다.

        按图像ID整理标注的辅助函数
        生成以图像ID为键、对应图像标注列表为值的字典。

        Returns:
            dict: 이미지 ID별 주석 매핑 딕셔너리
            dict: 按图像ID映射标注的字典
        """
        annotations_dict = {}
        for ann in self.annotations['annotations']:
            img_id = ann['image_id']
            if img_id not in annotations_dict:
                annotations_dict[img_id] = []
            annotations_dict[img_id].append(ann)
        return annotations_dict

    def __getitem__(self, idx):
        """데이터셋에서 단일 데이터를 가져오는 메서드
        인덱스를 기반으로 이미지와 해당 주석(바운딩 박스、라벨)을 반환합니다.

        从数据集中获取单条数据的方法
        根据索引返回图像及对应标注（边界框、标签）。

        Args:
            idx: 데이터를 가져올 인덱스
            idx: 要获取数据的索引

        Returns:
            tuple: (이미지 텐서, 주석 딕셔너리)
            tuple: (图像张量, 标注字典)
        """
        img_info = self.images[idx]
        # 이미지 파일 경로 생성 및 이미지 로드
        # 生成图像文件路径并加载图像
        img_path = os.path.join(self.root, img_info['file_name'])
        image = cv2.imread(img_path)
        # BGR(OpenCV 기본) → RGB 형식 변환 (PyTorch 기반 모델 호환)
        # BGR（OpenCV默认）→ RGB格式转换（适配PyTorch模型）
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        img_id = img_info['id']
        annotations = self.annotations_dict.get(img_id, [])  # 해당 이미지의 주석 가져오기 (获取对应图像的标注)

        # 바운딩 박스와 라벨 준비 (COCO 주석 형식: [xmin, ymin, width, height])
        # 准备边界框与标签（COCO标注格式：[xmin, ymin, width, height]）
        boxes = []
        labels = []
        for ann in annotations:
            xmin, ymin, width, height = ann['bbox']
            xmax = xmin + width  # xmax 계산 (计算xmax)
            ymax = ymin + height  # ymax 계산 (计算ymax)
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(ann['category_id'])

        # 텐서 형식으로 변환 (모델 입력에 맞춤)
        # 转换为张量格式（适配模型输入）
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # 주석 딕셔너리 생성 (Faster R-CNN 모델 입력 형식 준수)
        # 生成标注字典（遵循Faster R-CNN模型输入格式）
        target = {
            'boxes': boxes,          # 바운딩 박스 텐서 (边界框张量)
            'labels': labels,        # 라벨 텐서 (标签张量)
            'image_id': torch.tensor([img_id]),  # 이미지 ID (图像ID)
            'area': torch.as_tensor([ann['area'] for ann in annotations], dtype=torch.float32),  # 객체 면적 (物体面积)
            'iscrowd': torch.as_tensor([ann['iscrowd'] for ann in annotations], dtype=torch.int64)  # 군집 객체 여부 (是否为密集物体)
        }

        # 전처리 변환 적용 (있는 경우)
        # 应用预处理转换（如有）
        if self.transforms:
            image = self.transforms(image)

        return image, target

    def __len__(self):
        """데이터셋 총 크기를 반환하는 메서드
        데이터셋에 포함된 이미지 총 개수를 반환합니다.

        返回数据集总大小的方法
        返回数据集中包含的图像总数量。

        Returns:
            int: 데이터셋 크기 (이미지 개수)
            int: 数据集大小（图像数量）
        """
        return len(self.images)

# 사전 학습 객체 탐지 모델 로드 함수
# 加载预训练目标检测模型的函数
def load_detection_model(num_classes=len(DANGEROUS_CLASSES) + 1, pretrained=True):
    """객체 탐지 모델을 로드하고 초기화하는 함수
    Faster R-CNN ResNet50 FPN 모델을 로드하며，분류기를 사용자 정의 클래스 수에 맞게 교체합니다.

    加载并初始化目标检测模型的函数
    加载Faster R-CNN ResNet50 FPN模型，并根据自定义类别数量替换分类器。

    Args:
        num_classes: 총 클래스 수 (배경 포함，기본값: 위험물 클래스 수 + 1)
        num_classes: 总类别数（包含背景，默认值：危险物体类别数 + 1）
        pretrained: 사전 학습 가중치 사용 여부 (True=사용，False=미사용)
        pretrained: 是否使用预训练权重（True=使用，False=不使用）

    Returns:
        torch.nn.Module: 초기화된 객체 탐지 모델 (디바이스 할당 완료)
        torch.nn.Module: 初始化后的目标检测模型（已分配设备）
    """
    # 최신 사전 학습 가중치 로드 (PyTorch 1.13+ 권장 방식)
    # 加载最新预训练权重（PyTorch 1.13+推荐方式）
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT if pretrained else None
    model = fasterrcnn_resnet50_fpn(weights=weights)

    # 모델 분류기 교체 (사용자 정의 클래스 수에 맞춤)
    # 替换模型分类器（适配自定义类别数量）
    in_features = model.roi_heads.box_predictor.cls_score.in_features  # 입력 특징량 수 (输入特征量数)
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)  # 새로운 분류기 할당 (分配新分类器)

    model.to(device)  # 모델을 지정된 디바이스에 할당 (将模型分配到指定设备)
    return model

# 텍스트 생성 모델 로드 함수 (탐지 결과 설명 생성용)
# 加载文本生成模型的函数（用于生成检测结果解释）
def load_text_generator():
    """텍스트 생성 모델을 로드하는 함수
    GPT-2 모델을 기반으로 텍스트 생성 파이프라인을 초기화합니다.

    加载文本生成模型的函数
    基于GPT-2模型初始化文本生成流水线。

    Returns:
        transformers.pipeline: 텍스트 생성 파이프라인 (디바이스 할당 완료)
        transformers.pipeline: 文本生成流水线（已分配设备）
    """
    generator = pipeline(
        "text-generation",  # 작업 유형 (任务类型：文本生成)
        model="gpt2",       # 사용 모델 (使用的模型)
        # 디바이스 할당 (GPU:0，CPU:-1)
        device=0 if device.type == 'cuda' else -1,
        max_new_tokens=100  # 최대 생성 토큰 수 (最大生成token数)
    )
    return generator

# 모자이크 효과 적용 함수 (탐지된 위험물 영역 흐림 처리)
# 应用马赛克效果的函数（对检测到的危险物体区域模糊处理）
def apply_mosaic(image, bbox, mosaic_size=15):
    """탐지된 영역에 모자이크 효과를 적용하는 함수
    바운딩 박스로 지정된 영역에 모자이크 효과를 적용하여 이미지를 반환합니다.

    对检测区域应用马赛克效果的函数
    对边界框指定的区域应用马赛克效果，并返回处理后的图像。

    Args:
        image: 입력 이미지 (BGR/RGB 형식，NumPy 배열)
        image: 输入图像（BGR/RGB格式，NumPy数组）
        bbox: 바운딩 박스 좌표 (x1, y1, x2, y2)
        bbox: 边界框坐标（x1, y1, x2, y2）
        mosaic_size: 모자이크 크기 (값이 클수록 흐림 효과 강함，기본값:15)
        mosaic_size: 马赛克大小（值越大模糊效果越强，默认值：15）

    Returns:
        numpy.ndarray: 모자이크가 적용된 이미지
        numpy.ndarray: 应用马赛克后的图像
    """
    x1, y1, x2, y2 = map(int, bbox)
    # 바운딩 박스가 이미지 범위 내에 있도록 조정 (오버플로우 방지)
    # 调整边界框至图像范围内（防止溢出）
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)

    # ROI(Region of Interest) 추출 (관심 영역：위험물 영역)
    # 提取ROI（感兴趣区域：危险物体区域）
    roi = image[y1:y2, x1:x2]

    if roi.size == 0:  # 유효하지 않은 ROI인 경우 이미지 그대로 반환 (避免无效ROI处理)
        return image

    # 모자이크 효과 적용 (축소 → 확대 방식)
    # 应用马赛克效果（缩小→放大方式）
    roi_shape = roi.shape
    # ROI 축소 (모자이크 크기로 나눈 비율로 축소)
    # 缩小ROI（按马赛克大小比例缩小）
    roi = cv2.resize(roi,
                    (roi_shape[1]//mosaic_size, roi_shape[0]//mosaic_size),
                    interpolation=cv2.INTER_LINEAR)
    # ROI 확대 (원래 크기로 복원，인접 픽셀 복제로 흐림 효과)
    # 放大ROI（恢复原始大小，通过邻域像素复制实现模糊）
    roi = cv2.resize(roi,
                    (roi_shape[1], roi_shape[0]),
                    interpolation=cv2.INTER_NEAREST)

    # 처리된 ROI를 원본 이미지에 적용
    # 将处理后的ROI放回原始图像
    image[y1:y2, x1:x2] = roi
    return image

# 위험물 탐지 함수 (단일 프레임에서 위험물 찾기)
# 危险物体检测函数（在单帧中查找危险物体）
def detect_dangerous_objects(model, frame, score_threshold=0.5):
    """이미지 프레임에서 위험물을 탐지하는 함수
    모델을 사용하여 프레임에서 위험물을 탐지하고，신뢰도 기준을 통과한 객체를 반환합니다.

    在图像帧中检测危险物体的函数
    使用模型检测帧中的危险物体，并返回通过置信度标准的物体。

    Args:
        model: 객체 탐지 모델 (Faster R-CNN)
        model: 目标检测模型（Faster R-CNN）
        frame: 입력 이미지 프레임 (BGR/RGB 형식，NumPy 배열)
        frame: 输入图像帧（BGR/RGB格式，NumPy数组）
        score_threshold: 신뢰도 임계값 (값 이상인 객체만 유효，기본값:0.5)
        score_threshold: 置信度阈值（仅保留高于该值的物体，默认值：0.5）

    Returns:
        list: 탐지된 위험물 리스트 (각 요소는 객체 정보 딕셔너리)
        list: 检测到的危险物体列表（每个元素为物体信息字典）
    """
    # 프레임 전처리 (추론 모드 변환 적용)
    # 帧预处理（应用推理模式转换）
    transform = get_transform(train=False)
    frame_tensor = transform(frame).to(device)

    # 객체 탐지 (추론 모드로 실행，그라디언트 계산 비활성화)
    # 目标检测（推理模式运行，禁用梯度计算）
    model.eval()
    with torch.no_grad():
        predictions = model([frame_tensor])

    # 탐지 결과에서 위험물 필터링
    # 从检测结果中筛选危险物体
    dangerous_objects = []
    for pred in predictions:
        boxes = pred['boxes'].cpu().numpy()  # 바운딩 박스 (CPU로 이동，NumPy로 변환)
        labels = pred['labels'].cpu().numpy()  # 라벨 (CPU로 이동，NumPy로 변환)
        scores = pred['scores'].cpu().numpy()  # 신뢰도 (CPU로 이동，NumPy로 변환)

        for box, label, score in zip(boxes, labels, scores):
            # 위험물 카테고리에 속하고 신뢰도 임계값을 통과한 경우
            # 属于危险物体类别且通过置信度阈值的情况
            if label in DANGEROUS_CLASSES and score > score_threshold:
                dangerous_objects.append({
                    'bbox': box,          # 바운딩 박스 좌표 (边界框坐标)
                    'class': DANGEROUS_CLASSES[label],  # 위험물 종류 (危险物体种类)
                    'score': float(score)  # 신뢰도 (置信度)
                })

    return dangerous_objects

# 단일 프레임 처리 함수 (탐지 + 모자이크 적용)
# 单帧处理函数（检测+马赛克应用）
def process_frame(model, frame, score_threshold=0.5):
    """단일 이미지 프레임을 처리하는 함수
    프레임에서 위험물을 탐지하고，탐지된 영역에 모자이크 효과를 적용합니다.

    处理单张图像帧的函数
    检测帧中的危险物体，并对检测区域应用马赛克效果。

    Args:
        model: 객체 탐지 모델
        model: 目标检测模型
        frame: 입력 프레임 (BGR/RGB 형식，NumPy 배열)
        frame: 输入帧（BGR/RGB格式，NumPy数组）
        score_threshold: 신뢰도 임계값 (기본값:0.5)
        score_threshold: 置信度阈值（默认值：0.5）

    Returns:
        tuple: (모자이크 적용된 프레임, 탐지된 위험물 리스트)
        tuple: (应用马赛克后的帧, 检测到的危险物体列表)
    """
    # 위험물 탐지
    # 检测危险物体
    dangerous_objects = detect_dangerous_objects(model, frame, score_threshold)

    # 탐지된 위험물에 모자이크 적용
    # 对检测到的危险物体应用马赛克
    for obj in dangerous_objects:
        frame = apply_mosaic(frame, obj['bbox'])

    return frame, dangerous_objects

# 탐지 결과 설명 생성 함수 (자연어로 결과 설명)
# 生成检测结果解释的函数（用自然语言说明结果）
def generate_explanation(generator, detected_objects):
    """탐지된 위험물에 대한 자연어 설명을 생성하는 함수
    텍스트生成 모델을 사용하여，탐지 결과를 자연어로 설명합니다.

    生成危险物体检测结果自然语言解释的函数
    使用文本生成模型，将检测结果用自然语言描述。

    Args:
        generator: 텍스트 생성 파이프라인 (GPT-2 기반)
        generator: 文本生成流水线（基于GPT-2）
        detected_objects: 탐지된 위험물 리스트
        detected_objects: 检测到的危险物体列表

    Returns:
        str: 자연어 설명 문장
        str: 自然语言解释句子
    """
    if not detected_objects:  # 위험물이 탐지되지 않은 경우
        return "비디오에서 위험물이 감지되지 않았습니다."
        return "视频中未检测到危险物体。"

    # 탐지된 위험물 개수 통계
    # 统计检测到的危险物体数量
    object_counts = {}
    for obj in detected_objects:
        obj_class = obj['class']
        object_counts[obj_class] = object_counts.get(obj_class, 0) + 1

    # 텍스트 생성 프롬프트 작성
    # 编写文本生成提示词
    prompt = "비디오에서 다음 위험물이 감지되어 흐림 처리되었습니다："
    prompt = "视频中检测到以下危险物体并进行了模糊处理："
    for obj_class, count in object_counts.items():
        prompt += f" {count}개 {obj_class}，"

    prompt += "이 물체들은 위험물로 분류되어 안전 확보를 위해 흐림 처리되었습니다."
    prompt += "这些物体被归类为危险物品，为确保安全已进行模糊处理。"

    # 설명生成 (패딩 토큰 ID 지정으로 경고 방지)
    # 生成解释（指定填充token ID避免警告）
    explanation = generator(
        prompt,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=50256  # GPT-2 패딩 토큰 ID (GPT-2填充token ID)
    )[0]['generated_text']
    return explanation

# 전체 비디오 처리 함수 (프레임 단위 처리 + 결과 통합)
# 完整视频处理函数（帧级处理+结果整合）
def process_video(video_path, output_path, model, progress_callback=None):
    """전체 비디오 파일을 처리하는 함수
    비디오를 프레임 단위로 나누어 처리하고，처리된 프레임을 합쳐 새로운 비디오를 생성합니다.

    处理完整视频文件的函数
    将视频按帧拆分处理，合并处理后的帧生成新视频。

    Args:
        video_path: 입력 비디오 파일 경로
        video_path: 输入视频文件路径
        output_path: 출력 비디오 파일 경로
        output_path: 输出视频文件路径
        model: 객체 탐지 모델
        model: 目标检测模型
        progress_callback: 진행 상황 콜백 함수 (None=진행 상황 표시 안함)
        progress_callback: 进度回调函数（None=不显示进度）

    Returns:
        list: 비디오 전체에서 탐지된 위험물 리스트
        list: 视频中检测到的所有危险物体列表
    """
    # 비디오 로드 (MoviePy를 사용하여 비디오 읽기)
    # 加载视频（使用MoviePy读取视频）
    video = VideoFileClip(video_path)
    total_frames = int(video.fps * video.duration)  # 총 프레임 수 계산 (计算总帧数)
    processed_frames = 0  # 처리 완료된 프레임 수 (已处理帧数)

    # 전체 탐지 결과 저장 리스트
    # 存储完整检测结果的列表
    all_detected_objects = []

    def process_frame_wrapper(frame):
        """프레임 처리 래퍼 함수 (MoviePy fl_image와 호환)
        MoviePy의 fl_image 메서드가 호출하는 프레임 처리 함수입니다.

        帧处理包装函数（适配MoviePy的fl_image）
        供MoviePy的fl_image方法调用的帧处理函数。

        Args:
            frame: MoviePy 프레임 (RGB 형식，NumPy 배열)
            frame: MoviePy帧（RGB格式，NumPy数组）

        Returns:
            numpy.ndarray: 처리된 프레임 (RGB 형식)
            numpy.ndarray: 处理后的帧（RGB格式）
        """
        nonlocal processed_frames  # 외부 변수 참조 (引用外部变量)
        # RGB(MoviePy) → BGR(OpenCV) 변환 (프레임 처리에 필요)
        # RGB（MoviePy）→ BGR（OpenCV）转换（帧处理需要）
        frame_cv = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # 프레임 처리 (탐지 + 모자이크)
        # 帧处理（检测+马赛克）
        processed_frame, detected_objects = process_frame(model, frame_cv)

        # 탐지 결과 저장 (있는 경우)
        # 保存检测结果（如有）
        if detected_objects:
            all_detected_objects.extend(detected_objects)

        # 진행 상황 업데이트 (10프레임마다 콜백 호출，과도한 출력 방지)
        # 更新进度（每10帧调用一次回调，避免过度输出）
        processed_frames += 1
        if progress_callback and processed_frames % 10 == 0:
            progress = (processed_frames / total_frames) * 100
            progress_callback(progress)

        # BGR(OpenCV) → RGB(MoviePy) 변환 (비디오 합치기에 필요)
        # BGR（OpenCV）→ RGB（MoviePy）转换（视频合并需要）
        return cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)

    # 비디오 프레임 처리 및 새 비디오 생성 (H.264 코덱 사용，호환성 향상)
    # 处理视频帧并生成新视频（使用H.264编解码器，提升兼容性）
    processed_video = video.fl_image(process_frame_wrapper)
    processed_video.write_videofile(
        output_path,
        codec="libx264",  # H.264 코덱 (가장 널리 지원되는 코덱)
        verbose=False,    # 상세 로그 출력 안함 (不输出详细日志)
        logger=None       # 로거 비활성화 (禁用日志器)
    )

    return all_detected_objects

# 모델 학습 함수 (데이터셋 기반 모델 학습)
# 模型训练函数（基于数据集训练模型）
def train_model(model, train_dataset, val_dataset, params, output_dir='models'):
    """객체 탐지 모델을 학습하는 함수
    학습 데이터셋으로 모델을 학습하고，검증 데이터셋으로 성능을 평가하며，에폭마다 모델을 저장합니다.

    训练目标检测模型的函数
    使用训练数据集训练模型，用验证数据集评估性能，每轮次保存模型。

    Args:
        model: 학습할 객체 탐지 모델
        model: 待训练的目标检测模型
        train_dataset: 학습 데이터셋 (DangerousObjectsDataset 인스턴스)
        train_dataset: 训练数据集（DangerousObjectsDataset实例）
        val_dataset: 검증 데이터셋 (DangerousObjectsDataset 인스턴스)
        val_dataset: 验证数据集（DangerousObjectsDataset实例）
        params: 학습 파라미터 딕셔너리 (TRAIN_PARAMS 참조)
        params: 训练参数字典（参考TRAIN_PARAMS）
        output_dir: 모델 저장 디렉토리 경로 (기본값: 'models')
        output_dir: 模型保存目录路径（默认值：'models'）

    Returns:
        torch.nn.Module: 학습 완료된 모델
        torch.nn.Module: 训练完成的模型
    """
    # 데이터 로더 생성 (배치 처리 및 셔플링 지원)
    # 创建数据加载器（支持批量处理与洗牌）
    train_loader = DataLoader(
        train_dataset,
        batch_size=params['batch_size'],
        shuffle=True,  # 학습 데이터는 셔플링 (训练数据洗牌)
        num_workers=params['workers'],
        # 데이터 병합 함수 (이미지/주석 길이가 다를 경우 처리)
        # 数据合并函数（处理图像/标注长度不一致的情况）
        collate_fn=lambda x: tuple(zip(*x))
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=params['batch_size'],
        shuffle=False,  # 검증 데이터는 셔플링 안함 (验证数据不洗牌)
        num_workers=params['workers'],
        collate_fn=lambda x: tuple(zip(*x))
    )

    # 최적화기 및 학습률 스케줄러 설정
    # 设置优化器与学习率调度器
    # 학습 가능한 파라미터만 필터링 (仅筛选可训练参数)
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(
        trainable_params,
        lr=params['lr'],
        momentum=params['momentum'],
        weight_decay=params['weight_decay']
    )

    # 단계별 학습률 감소 스케줄러 (StepLR)
    # 阶梯式学习率衰减调度器（StepLR）
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=params['lr_step_size'],
        gamma=params['lr_gamma']
    )

    # TensorBoard 로거 초기화 (학습 과정 시각화)
    # 初始化TensorBoard日志器（训练过程可视化）
    log_dir = os.path.join('runs', datetime.now().strftime('%Y%m%d-%H%M%S'))
    writer = SummaryWriter(log_dir)

    # 모델 저장 디렉토리 생성 (없는 경우)
    # 创建模型保存目录（不存在时）
    os.makedirs(output_dir, exist_ok=True)

    # 학습 루프 (에폭 단위로 반복)
    # 训练循环（按轮次迭代）
    for epoch in range(params['num_epochs']):
        print(f"에폭 {epoch+1}/{params['num_epochs']}")
        print(f"轮次 {epoch+1}/{params['num_epochs']}")
        print('-' * 20)

        # 1. 학습 단계 (Training Phase)
        # 1. 训练阶段（Training Phase）
        model.train()  # 모델을 학습 모드로 설정 (将模型设为训练模式)
        train_loss = 0.0  # 총 학습 손실 (总训练损失)

        # 학습 데이터 로더를 통한 배치 처리 (진행 상황 바 표시)
        # 通过训练数据加载器处理批量数据（显示进度条）
        for images, targets in tqdm(train_loader, desc='학습 중...'):
            for images, targets in tqdm(train_loader, desc='训练中...'):
                # 이미지와 주석을 디바이스에 할당 (GPU/CPU)
                # 将图像和标注分配到设备（GPU/CPU）
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                # 순전파 (forward pass) → 손실 계산
                # 前向传播（forward pass）→ 计算损失
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())  # 총 손실 합산 (总损失求和)

                # 역전파 (backward pass) → 최적화
                # 反向传播（backward pass）→ 优化
                optimizer.zero_grad()  # 그라디언트 초기화 (梯度初始化)
                losses.backward()      # 그라디언트 계산 (梯度计算)
                optimizer.step()       # 파라미터 업데이트 (参数更新)

                train_loss += losses.item()  # 손실 누적 (累积损失)

        # 평균 학습 손실 계산 및 로깅
        # 计算平均训练损失并记录
        avg_train_loss = train_loss / len(train_loader)
        writer.add_scalar('Loss/Train', avg_train_loss, epoch)  # TensorBoard에 로그 추가
        print(f"평균 학습 손실: {avg_train_loss:.4f}")
        print(f"平均训练损失: {avg_train_loss:.4f}")

        # 2. 검증 단계 (Validation Phase)
        # 2. 验证阶段（Validation Phase）
        model.eval()  # 모델을 추론 모드로 설정 (将模型设为推理模式)
        val_loss = 0.0  # 총 검증 손실 (总验证损失)

        # 그라디언트 계산 비활성화 (검증 시 불필요한 계산 방지)
        # 禁用梯度计算（避免验证时不必要的计算）
        with torch.no_grad():
            for images, targets in tqdm(val_loader, desc='검증 중...'):
                for images, targets in tqdm(val_loader, desc='验证中...'):
                    images = list(image.to(device) for image in images)
                    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                    # 순전파 → 손실 계산 (역전파 하지 않음)
                    # 前向传播 → 计算损失（不反向传播）
                    loss_dict = model(images, targets)
                    losses = sum(loss for loss in loss_dict.values())
                    val_loss += losses.item()

        # 평균 검증 손실 계산 및 로깅
        # 计算平均验证损失并记录
        avg_val_loss = val_loss / len(val_loader)
        writer.add_scalar('Loss/Validation', avg_val_loss, epoch)  # TensorBoard에 로그 추가
        print(f"평균 검증 손실: {avg_val_loss:.4f}")
        print(f"平均验证损失: {avg_val_loss:.4f}")

        # 학습률 업데이트 (스케줄러에 따라)
        # 根据调度器更新学习率
        lr_scheduler.step()

        # 에폭마다 모델 저장 (가중치만 저장，메모리 효율적)
        # 每轮次保存模型（仅保存权重，内存高效）
        model_path = os.path.join(output_dir, f'model_epoch_{epoch+1}.pth')
        torch.save(model.state_dict(), model_path)
        print(f"에폭 {epoch+1} 모델 저장 완료: {model_path}")
        print(f"轮次 {epoch+1} 模型保存完成: {model_path}")

    writer.close()  # TensorBoard 로거 닫기 (关闭TensorBoard日志器)
    print("모델 학습 완료!")
    print("模型训练完成!")
    return model

# 모델 평가 함수 (정확도/재현율/F1 스코어 계산)
# 模型评估函数（计算精确率/召回率/F1分数）
def evaluate_model(model, dataset):
    """학습된 모델을 평가하는 함수
    데이터셋을 사용하여 모델의 정확도(Precision)、재현율(Recall)、F1 스코어를 계산합니다.

    评估训练后模型的函数
    使用数据集计算模型的精确率（Precision）、召回率（Recall）、F1分数。

    Args:
        model: 평가할 객체 탐지 모델
        model: 待评估的目标检测模型
        dataset: 평가용 데이터셋 (DangerousObjectsDataset 인스턴스)
        dataset: 评估用数据集（DangerousObjectsDataset实例）

    Returns:
        dict: 평가 지표 딕셔너리 (precision, recall, f1)
        dict: 评估指标字典（精确率、召回率、F1分数）
    """
    model.eval()  # 모델을 추론 모드로 설정 (将模型设为推理模式)
    # 평가 데이터 로더 생성 (배치 크기=1，정확한 개수 계산 위해)
    # 创建评估数据加载器（批量大小=1，为精确计算数量）
    data_loader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        collate_fn=lambda x: tuple(zip(*x))
    )

    # 평가 지표 계산을 위한 변수 초기화
    # 初始化评估指标计算用变量
    total_objects = 0    # 실제 위험물 총 개수 (实际危险物体总数)
    detected_objects = 0 # 모델이 탐지한 객체 총 개수 (模型检测到的物体总数)
    correct_detections = 0  # 정확히 탐지된 위험물 개수 (正确检测的危险物体总数)

    # 그라디언트 계산 비활성화
    # 禁用梯度计算
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc='평가 중...'):
            for images, targets in tqdm(data_loader, desc='评估中...'):
                images = list(image.to(device) for image in images)
                outputs = model(images)  # 모델 추론 결과 (模型推理结果)

                # 각 이미지의 실제 주석과 예측 결과 비교
                # 比较每张图像的实际标注与预测结果
                for output, target in zip(outputs, targets):
                    true_labels = target['labels'].numpy()  # 실제 라벨 (实际标签)
                    pred_labels = output['labels'].cpu().numpy()  # 예측 라벨 (预测标签)
                    pred_scores = output['scores'].cpu().numpy()  # 예측 신뢰도 (预测置信度)

                    # 신뢰도 0.5 이상인 예측만 유효하게 처리
                    # 仅处理置信度0.5以上的预测结果
                    valid_indices = pred_scores > 0.5
                    pred_labels = pred_labels[valid_indices]

                    # 개수 업데이트
                    # 更新计数
                    total_objects += len(true_labels)
                    detected_objects += len(pred_labels)

                    # 정확히 탐지된 개수 계산 (예측 라벨이 실제 라벨에 포함되는 경우)
                    # 计算正确检测数量（预测标签属于实际标签的情况）
                    for label in pred_labels:
                        if label in true_labels:
                            correct_detections += 1

    # 평가 지표 계산 (0으로 나누는 오류 방지)
    # 计算评估指标（防止除以零错误）
    precision = correct_detections / detected_objects if detected_objects > 0 else 0.0
    recall = correct_detections / total_objects if total_objects > 0 else 0.0
    # F1 스코어 (정확도와 재현율의 조화 평균)
    # F1分数（精确率与召回率的调和平均）
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # 평가 결과 출력
    # 输出评估结果
    print("\n=== 모델 평가 결과 ===")
    print("\n=== 模型评估结果 ===")
    print(f"정확도 (Precision): {precision:.4f}")
    print(f"精确率 (Precision): {precision:.4f}")
    print(f"재현율 (Recall): {recall:.4f}")
    print(f"召回率 (Recall): {recall:.4f}")
    print(f"F1 스코어: {f1:.4f}")
    print(f"F1 分数: {f1:.4f}")

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# 메인 함수 (시스템 제어 및 사용자 인터랙션)
# 主函数（系统控制与用户交互）
def main():
    """메인 함수
    시스템의 전체 흐름을 제어하며，사용자로부터 입력을 받아 학습 또는 추론 모드를 실행합니다.

    主函数
    控制系统整体流程，接收用户输入并执行训练或推理模式。
    """
    print("=== 비디오 위험물 탐지 및 흐림 처리 시스템 ===")
    print("=== 视频危险物体检测及模糊处理系统 ===")

    # 사용자로부터 모드 선택 입력 받기 (학습/추론)
    # 接收用户输入的模式选择（训练/推理）
    mode = input("모드를 선택하세요 (train:학습 / infer:추론): ").strip().lower()
    mode = input("请选择模式（train:训练 / infer:推理）: ").strip().lower()

    # 객체 탐지 모델 초기화
    # 初始化目标检测模型
    print("\n객체 탐지 모델 로딩 중...")
    print("\n目标检测模型加载中...")
    # 총 클래스 수 = 위험물 클래스 수 + 1 (배경 클래스)
    # 总类别数 = 危险物体类别数 + 1（背景类别）
    num_classes = len(DANGEROUS_CLASSES) + 1
    detection_model = load_detection_model(num_classes=num_classes)

    if mode == 'train':
        # 1. 학습 모드 (Training Mode)
        # 1. 训练模式（Training Mode）
        print("\n--- 학습 모드 실행 ---")
        print("\n--- 执行训练模式 ---")

        # 학습 데이터 준비 안내
        # 训练数据准备说明
        print("학습 데이터를 업로드해주세요.")
        print("请上传训练数据。")
        print("주의: 데이터셋은 COCO 형식 주석 파일을 포함해야 합니다.")
        print("注意：数据集需包含COCO格式的标注文件。")

        # 1.1 이미지 ZIP 파일 업로드 및 압축 해제
        # 1.1 上传图像ZIP文件并解压
        print("\n1. 이미지 디렉토리 ZIP 파일 업로드:")
        print("\n1. 上传图像目录ZIP文件:")
        image_zip = files.upload()
        image_zip_path = next(iter(image_zip))  # 업로드된 첫 번째 파일 경로 (上传的第一个文件路径)
        # ZIP 파일을 dataset/images 디렉토리에 압축 해제 (무음 모드)
        # 将ZIP文件解压到dataset/images目录（静默模式）
        !unzip -q {image_zip_path} -d dataset/images

        # 1.2 학습 주석 파일 업로드
        # 1.2 上传训练标注文件
        print("\n2. 학습용 주석 파일 (JSON) 업로드:")
        print("\n2. 上传训练用标注文件（JSON）:")
        train_ann = files.upload()
        train_ann_path = next(iter(train_ann))

        # 1.3 검증 주석 파일 업로드
        # 1.3 上传验证标注文件
        print("\n3. 검증용 주석 파일 (JSON) 업로드:")
        print("\n3. 上传验证用标注文件（JSON）:")
        val_ann = files.upload()
        val_ann_path = next(iter(val_ann))

        # 1.4 데이터셋 인스턴스 생성
        # 1.4 创建数据集实例
        train_dataset = DangerousObjectsDataset(
            root='dataset/images',
            annotations_file=train_ann_path,
            transforms=get_transform(train=True)  # 학습용 전처리 (训练用预处理)
        )

        val_dataset = DangerousObjectsDataset(
            root='dataset/images',
            annotations_file=val_ann_path,
            transforms=get_transform(train=False)  # 추론용 전처리 (推理用预处理)
        )

        # 데이터셋 크기 출력
        # 输出数据集大小
        print(f"\n학습 데이터 개수: {len(train_dataset)}, 검증 데이터 개수: {len(val_dataset)}")
        print(f"\n训练数据数量: {len(train_dataset)}, 验证数据数量: {len(val_dataset)}")

        # 1.5 모델 학습 실행
        # 1.5 执行模型训练
        print("\n모델 학습 시작...")
        print("\n开始模型训练...")
        trained_model = train_model(
            detection_model,
            train_dataset,
            val_dataset,
            TRAIN_PARAMS
        )

        # 1.6 모델 평가
        # 1.6 模型评估
        print("\n학습된 모델 평가 중...")
        print("\n评估训练后的模型...")
        evaluate_model(trained_model, val_dataset)

        # 1.7 최종 모델 다운로드
        # 1.7 下载最终模型
        print("\n최종 모델 다운로드 준비 중...")
        print("\n准备下载最终模型...")
        final_model_path = os.path.join('models', f'model_epoch_{TRAIN_PARAMS["num_epochs"]}.pth')
        files.download(final_model_path)
        print("최종 모델 다운로드 완료!")
        print("最终模型下载完成!")

    elif mode == 'infer':
        # 2. 추론 모드 (Inference Mode)
        # 2. 推理模式（Inference Mode）
        print("\n--- 추론 모드 실행 ---")
        print("\n--- 执行推理模式 ---")

        # 사용자 정의 모델 사용 여부 입력 받기
        # 接收是否使用自定义模型的输入
        use_custom_model = input("학습 완료된 사용자 정의 모델을 사용하시겠습니까? (y/n): ").strip().lower() == 'y'
        use_custom_model = input("是否使用训练完成的自定义模型？(y/n): ").strip().lower() == 'y'

        if use_custom_model:
            # 사용자 정의 모델 업로드 및 로드
            # 上传并加载自定义模型
            print("\n사용자 정의 모델 파일 (.pth) 업로드:")
            print("\n上传自定义模型文件（.pth）:")
            custom_model = files.upload()
            model_path = next(iter(custom_model))
            # 모델 가중치 로드 (디바이스에 맞춰 자동 할당)
            # 加载模型权重（自动适配设备）
            detection_model.load_state_dict(torch.load(model_path, map_location=device))
            detection_model.to(device)
            print("사용자 정의 모델 로딩 완료!")
            print("自定义模型加载完成!")

        # 2.1 텍스트生成 모델 로드
        # 2.1 加载文本生成模型
        print("\n텍스트 생성 모델 로딩 중...")
        print("\n文本生成模型加载中...")
        text_generator = load_text_generator()

        # 2.2 처리할 비디오 업로드
        # 2.2 上传待处理视频
        print("\n처리할 비디오 파일 업로드:")
        print("\n上传待处理视频文件:")
        uploaded = files.upload()
        video_filename = next(iter(uploaded))  # 업로드된 비디오 파일명 (上传的视频文件名)

        # 2.3 비디오 처리 실행
        # 2.3 执行视频处理
        print("\n비디오 처리 시작...")
        print("\n开始处理视频...")
        output_filename = "processed_video.mp4"  # 처리 후 출력 파일명 (处理后输出文件名)

        # 진행 상황 콜백 함수 (사용자에게 진행률 표시)
        # 进度回调函数（向用户显示进度）
        def progress_callback(progress):
            print(f"처리 진행률: {progress:.1f}%", end='\r')
            print(f"处理进度: {progress:.1f}%", end='\r')

        # 비디오 처리 및 탐지 결과 수집
        # 处理视频并收集检测结果
        detected_objects = process_video(
            video_filename,
            output_filename,
            detection_model,
            progress_callback
        )
        print("\n비디오 처리 완료!")
        print("\n视频处理完成!")

        # 2.4 탐지 결과 설명生成
        # 2.4 生成检测结果解释
        print("\n탐지 결과 설명 생성 중...")
        print("\n生成检测结果解释中...")
        explanation = generate_explanation(text_generator, detected_objects)
        print("\n=== 처리 결과 설명 ===")
        print("\n=== 处理结果解释 ===")
        print(explanation)

        # 2.5 처리된 비디오 다운로드
        # 2.5 下载处理后的视频
        print("\n처리된 비디오 다운로드 준비 중...")
        print("\n准备下载处理后的视频...")
        files.download(output_filename)
        print("처리된 비디오 다운로드 완료!")
        print("处理后的视频下载完成!")

    else:
        # 유효하지 않은 모드 입력 시 오류 메시지 출력
        # 输入无效模式时输出错误信息
        print("\n오류: 유효하지 않은 모드입니다. 'train' 또는 'infer'를 입력해주세요.")
        print("\n错误：无效模式。请输入'train'或'infer'。")

# 스크립트 실행 시 메인 함수 호출
# 脚本执行时调用主函数
if __name__ == "__main__":
    main()